In [35]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit

qr = QuantumRegister(2, name='qr')
cr = ClassicalRegister(2, name='cr')
qc = QuantumCircuit(qr, cr, name='qc')
# TIMESTAMP: 1738243262.011104

# Apply gate operations
# <START_GATES>

qc.x(qr[0])
qc.z(qr[0])
# qc.x(1) # EXTRA
# qc.rx(4.695018, qr[6])
# qc.s(qr[7])
# qc.cry(2.828155, qr[9], qr[5])
# qc.t(qr[2])

qc.measure(qr, cr)

print(qc.draw())

      ┌───┐┌───┐┌─┐
qr_0: ┤ X ├┤ Z ├┤M├
      └┬─┬┘└───┘└╥┘
qr_1: ─┤M├───────╫─
       └╥┘       ║ 
cr: 2/══╩════════╩═
        1        0 


In [36]:
from copy import deepcopy
from pytket.circuit import OpType
from pytket.passes import (
    FullPeepholeOptimise, PeepholeOptimise2Q, RemoveRedundancies,
    EulerAngleReduction, KAKDecomposition,
    CliffordPushThroughMeasures, FlattenRegisters,
    PauliSimp, GreedyPauliSimp,
    OptimisePhaseGadgets,
    ZXGraphlikeOptimisation
)
from pytket.qasm import circuit_to_qasm_str
from pytket.circuit.display import render_circuit_jupyter
from pytket.extensions.qiskit import qiskit_to_tk, tk_to_qiskit

tket_circ = qiskit_to_tk(qc.decompose().decompose())


optimization_passes = {
    "FullPeepholeOptimise": FullPeepholeOptimise(),
    # "PeepholeOptimise2Q": PeepholeOptimise2Q(),
    # "RemoveRedundancies": RemoveRedundancies(),
    # "EulerAngleReduction": EulerAngleReduction(q=OpType.Rz, p=OpType.Rx),
    # "KAKDecomposition": KAKDecomposition(),
    # "CliffordPushThroughMeasures": CliffordPushThroughMeasures(),
    #     "FlattenRegisters": FlattenRegisters(),
    #     "PauliSimp": PauliSimp(),
    #     "GreedyPauliSimp": GreedyPauliSimp(),
    #     "OptimisePhaseGadgets": OptimisePhaseGadgets(),
    #     "ZXGraphlikeOptimisation": ZXGraphlikeOptimisation()
}

for opt_pass_name, optimization_pass in optimization_passes.items():
    print(f"Applying {opt_pass_name}")
    i_qc = deepcopy(tket_circ)
    optimization_pass.apply(i_qc)
    qc_opt = tk_to_qiskit(i_qc)
    # render_circuit_jupyter(i_qc)
    # i_opt_circuit_qasm = circuit_to_qasm_str(
    #     i_qc, header="hqslib1", maxwidth=200)
    # i_qc, header="qelib1", maxwidth=200)
    print(qc_opt.draw())

Applying FullPeepholeOptimise
global phase: π/2
      ┌───────────────┐┌─┐
qr_0: ┤ U(π,-π/2,π/2) ├┤M├
      └──────┬─┬──────┘└╥┘
qr_1: ───────┤M├────────╫─
             └╥┘        ║ 
cr: 2/════════╩═════════╩═
              1         0 


In [37]:
from mqt import qcec
result = qcec.verify(
    qc,
    qc_opt,
    transform_dynamic_circuit=True)


def compare_unitary(qc1, qc2) -> bool:
    # remove the measure gates
    qc1 = qc1.copy()
    qc2 = qc2.copy()
    qc1.data = [gate for gate in qc1.data if gate[0].name != 'measure']
    qc2.data = [gate for gate in qc2.data if gate[0].name != 'measure']
    print(qc1.draw())
    print(qc2.draw())

    # compare the unitary matrices
    from qiskit.quantum_info import Operator
    u1 = Operator(qc1).data
    u2 = Operator(qc2).data
    from numpy import allclose
    circuits_are_equivalent = allclose(u1, u2)
    if not circuits_are_equivalent:
        print("unitary matrices are not equivalent")
        print(u1)
        print(u2)
    return circuits_are_equivalent


unitary_equivalence = compare_unitary(qc, qc_opt)
print("unitary_equivalence:")
print(unitary_equivalence)


equivalence = str(result.equivalence)
print("equivalence:")
print(equivalence)

      ┌───┐┌───┐
qr_0: ┤ X ├┤ Z ├
      └───┘└───┘
qr_1: ──────────
                
cr: 2/══════════
                
global phase: π/2
      ┌───────────────┐
qr_0: ┤ U(π,-π/2,π/2) ├
      └───────────────┘
qr_1: ─────────────────
                       
cr: 2/═════════════════
                       
unitary matrices are not equivalent
[[ 0.+0.j  1.+0.j  0.+0.j  0.+0.j]
 [-1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  1.+0.j]
 [ 0.+0.j  0.+0.j -1.+0.j  0.+0.j]]
[[3.74939946e-33+6.1232340e-17j 1.00000000e+00-1.2246468e-16j
  0.00000000e+00+0.0000000e+00j 0.00000000e+00+0.0000000e+00j]
 [1.00000000e+00+0.0000000e+00j 3.74939946e-33+6.1232340e-17j
  0.00000000e+00+0.0000000e+00j 0.00000000e+00+0.0000000e+00j]
 [0.00000000e+00+0.0000000e+00j 0.00000000e+00+0.0000000e+00j
  3.74939946e-33+6.1232340e-17j 1.00000000e+00-1.2246468e-16j]
 [0.00000000e+00+0.0000000e+00j 0.00000000e+00+0.0000000e+00j
  1.00000000e+00+0.0000000e+00j 3.74939946e-33+6.1232340e-17j]]
unitary_equivalen

/tmp/ipykernel_1085421/146943001.py:12: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 2.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  qc1.data = [gate for gate in qc1.data if gate[0].name != 'measure']
/tmp/ipykernel_1085421/146943001.py:13: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 2.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  qc2.data = [gate for gate in qc2.data if gate[0].name != 'measure']


In [40]:
from qiskit.transpiler.passes import (
    Collect2qBlocks,
    ConsolidateBlocks,
    UnitarySynthesis,
    RemoveFinalMeasurements
)
from qiskit.transpiler import PassManager
from termcolor import colored
from qiskit.quantum_info import Statevector
circuit_a = qc
circuit_b = qc_opt

# remove any final measurements
circuit_a = deepcopy(circuit_a)
circuit_b = deepcopy(circuit_b)


basis_gates = ["rx", "ry", "rxx"]
translate = PassManager(
    [
        # Collect2qBlocks(),
        # ConsolidateBlocks(basis_gates=basis_gates),
        # UnitarySynthesis(basis_gates),
        RemoveFinalMeasurements()
    ]
)
circuit_a = translate.run(circuit_a)
circuit_b = translate.run(circuit_b)


sv_a = Statevector.from_instruction(circuit_a)
print(colored("Statevector A: ", "green"), sv_a)

sv_b = Statevector.from_instruction(circuit_b)
print(colored("Statevector B: ", "blue"), sv_b)

# compare the statevectors
probabilities_a = sv_a.probabilities_dict()
print(colored("Probabilities A: ", "green"), probabilities_a)
probabilities_b = sv_b.probabilities_dict()
print(colored("Probabilities B: ", "blue"), probabilities_b)

Statevector A:  Statevector([ 0.+0.j, -1.+0.j,  0.+0.j, -0.+0.j],
            dims=(2, 2))
Statevector B:  Statevector([3.74939946e-33+6.123234e-17j, 1.00000000e+00+0.000000e+00j,
             0.00000000e+00+0.000000e+00j, 0.00000000e+00+0.000000e+00j],
            dims=(2, 2))
Probabilities A:  {'01': 1.0}
Probabilities B:  {'00': 3.749399456654644e-33, '01': 1.0}


In [15]:
import numpy as np
from qiskit.quantum_info import Operator
qc_start = QuantumCircuit(2)
qc.p(4.465694, 0)

qc_empty = QuantumCircuit(2)


op_start = Operator(qc_start)
unitary_start = op_start.data
print("unitary_start:")
print(unitary_start)

op_empty = Operator(qc_empty)
unitary_empty = op_empty.data
print("unitary_empty:")
print(unitary_empty)

print("unitary_start - unitary_empty:")
# all close
print(np.allclose(unitary_start, unitary_empty))

unitary_start:
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]]
unitary_empty:
[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]]
unitary_start - unitary_empty:
True


In [17]:
from mqt import qcec
result = qcec.verify(
    qc_start,
    qc_empty,
    transform_dynamic_circuit=True)
equivalence = str(result.equivalence)
print("equivalence:")
print(equivalence)

equivalence:
equivalent


In [13]:
from copy import deepcopy
from pytket.circuit import OpType
from pytket.passes import (
    FullPeepholeOptimise, PeepholeOptimise2Q, RemoveRedundancies,
    EulerAngleReduction, KAKDecomposition,
    CliffordPushThroughMeasures, FlattenRegisters,
    PauliSimp, GreedyPauliSimp,
    OptimisePhaseGadgets,
    ZXGraphlikeOptimisation
)
from pytket.qasm import circuit_to_qasm_str
from pytket.circuit.display import render_circuit_jupyter
from pytket.extensions.qiskit import qiskit_to_tk

tket_circ = qiskit_to_tk(qc.decompose().decompose())


optimization_passes = {
    # "FullPeepholeOptimise": FullPeepholeOptimise(),
    # "PeepholeOptimise2Q": PeepholeOptimise2Q(),
    "RemoveRedundancies": RemoveRedundancies(),
    # "EulerAngleReduction": EulerAngleReduction(q=OpType.Rz, p=OpType.Rx),
    # "KAKDecomposition": KAKDecomposition(),
    # "CliffordPushThroughMeasures": CliffordPushThroughMeasures(),
    #     "FlattenRegisters": FlattenRegisters(),
    #     "PauliSimp": PauliSimp(),
    #     "GreedyPauliSimp": GreedyPauliSimp(),
    #     "OptimisePhaseGadgets": OptimisePhaseGadgets(),
    #     "ZXGraphlikeOptimisation": ZXGraphlikeOptimisation()
}

for opt_pass_name, optimization_pass in optimization_passes.items():
    print(f"Applying {opt_pass_name}")
    i_qc = deepcopy(tket_circ)
    optimization_pass.apply(i_qc)
    render_circuit_jupyter(i_qc)
    # i_opt_circuit_qasm = circuit_to_qasm_str(
    #     i_qc, header="hqslib1", maxwidth=200)
    # i_qc, header="qelib1", maxwidth=200)

Applying RemoveRedundancies
